In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)
import pickle

In [2]:
data_dir = '/opt/ml/project/data/' # 경로는 상황에 맞춰서 수정해주세요!
csv_file_path = os.path.join(data_dir, 'total_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
df = pd.read_csv(csv_file_path, parse_dates=['Timestamp']) 
df = df.sort_values(by=['userID','Timestamp']).reset_index(drop=True)

In [3]:
elapsed_time = df.Timestamp.shift(-1) - df.Timestamp
elapsed_time = elapsed_time.values / np.timedelta64(1, 's')
df['elapsedTime'] = elapsed_time
df.loc[df.groupby('userID', as_index=False).nth(-1).index, 'elapsedTime'] = np.NaN

In [4]:
df['test_cumcount'] = df.groupby(['userID', 'testId']).cumcount() + 1
temp_df = df.groupby(['userID', 'testId'],as_index=False).assessmentItemID.nunique()
temp_df.columns = ['userID', 'testId', 'test_itemnum']
temp_df
df = pd.merge(df, temp_df, how='left', on=['userID', 'testId'])
df['test_num'] = (df['test_cumcount'] - 1) // df['test_itemnum']

In [5]:
temp_df = df[df['test_cumcount'] % df['test_itemnum'] == 1]
temp_df = temp_df[['userID', 'testId', 'test_num', 'Timestamp']]
temp_df.columns = ['userID', 'testId', 'test_num', 'test_start_time']
df = pd.merge(df, temp_df, how='left', on=['userID', 'testId', 'test_num'])

temp_df = df[df['test_cumcount'] % df['test_itemnum'] == 0]
temp_df = temp_df[['userID', 'testId', 'test_num', 'Timestamp']]
temp_df.columns = ['userID', 'testId', 'test_num', 'test_end_time']
df = pd.merge(df, temp_df, how='left', on=['userID', 'testId', 'test_num'])

In [6]:
cond1 = df.userID.shift(-1) == df.userID
cond2_1 = df.test_start_time < df.test_start_time.shift(-1)
cond2_2 = df.test_end_time > df.test_start_time.shift(-1)
cond3 = df.test_cumcount % df.test_itemnum == 0
cond4 = df.elapsedTime > 60

df.loc[~(cond1&cond2_1&cond2_2)&cond3&cond4, 'elapsedTime'] = np.NaN

In [7]:
df['testType'] = df['testId'].apply(lambda x:x[:3])
df['testNumber'] = df['testId'].apply(lambda x:x[-3:])
df['questionNumber'] = df['assessmentItemID'].apply(lambda x:x[-3:])

In [9]:
csv_save_path = os.path.join(data_dir, 'total_data_v2.csv')
df.to_csv(csv_save_path, index=False)

In [28]:
data_dir = '/opt/ml/project/data/'
csv_file_path = os.path.join(data_dir, 'total_data_v2.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
df = pd.read_csv(csv_file_path, parse_dates=['Timestamp', 'test_start_time', 'test_end_time']) 
df = df.sort_values(by=['userID','Timestamp']).reset_index(drop=True)
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsedTime,test_cumcount,test_itemnum,test_num,test_start_time,test_end_time,testType,testNumber,questionNumber
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,3.0,1,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,8.0,2,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,2
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,7.0,3,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,3
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7.0,4,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,4
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,11.0,5,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,NaN,5,5,0,2020-06-05 06:46:41,2020-06-05 06:50:21,A03,71,5
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,11.0,1,4,0,2020-08-21 01:06:39,2020-08-21 01:08:49,A04,165,1
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,46.0,2,4,0,2020-08-21 01:06:39,2020-08-21 01:08:49,A04,165,2
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,73.0,3,4,0,2020-08-21 01:06:39,2020-08-21 01:08:49,A04,165,3


In [30]:
for offset in [600, 1200, 3600, 6000, 12000, 86400]:
    time_cond = df[f'elapsedTime'] < offset

    # # version 1
    # elapsedTime_arr = df['elapsedTime'].values
    # elapsedTime_arr[~time_cond] = df[time_cond].elapsedTime.mean()
    # df[f'elapsedTime{offset}'] = elapsedTime_arr

    # df[time_cond].elapsedTime.mean()

    # version 2
    df[f'elapsedTime{offset}'] = df['elapsedTime'].values
    question_time_dict = df[time_cond].groupby('assessmentItemID').elapsedTime.mean().to_dict()
    global_elapsedTime_mean = df.loc[~time_cond, 'elapsedTime'].mean()
    df.loc[~time_cond, f'elapsedTime{offset}'] = df.loc[~time_cond, 'assessmentItemID'].apply(lambda x:question_time_dict[x] if x in question_time_dict.keys() else global_elapsedTime_mean)

In [34]:
print(df.elapsedTime600.mean())
print(df.elapsedTime1200.mean())
print(df.elapsedTime3600.mean())
print(df.elapsedTime6000.mean())
print(df.elapsedTime12000.mean())
print(df.elapsedTime86400.mean())

52.50221717096527
59.95774798942759
70.35961014428337
75.19269088257148
83.02760473421414
122.23937009066915


In [37]:
df.groupby(['userID'])[f'elapsedTime'].rolling(3, min_periods=1).mean().values

array([        nan,         nan,  6.        , ...,         nan,
       43.33333333,         nan])

In [23]:
df[df['assessmentItemID'] == 'A040165004']

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsedTime,test_cumcount,test_itemnum,test_num,test_start_time,test_end_time,testType,testNumber,questionNumber
1289,1,A040165004,A040000165,0,2020-07-26 23:02:54,8836,NaN,4,4,0,2020-07-26 22:58:44,2020-07-26 23:02:54,A04,165,4
1482,1,A040165004,A040000165,1,2020-08-28 13:50:13,8836,NaN,8,4,1,2020-08-28 13:47:10,2020-08-28 13:50:13,A04,165,4
7439,9,A040165004,A040000165,0,2020-10-07 06:44:19,8836,NaN,4,4,0,2020-10-07 06:42:53,2020-10-07 06:44:19,A04,165,4
15016,17,A040165004,A040000165,0,2020-09-15 12:38:10,8836,NaN,4,4,0,2020-09-15 12:36:12,2020-09-15 12:38:10,A04,165,4
30089,33,A040165004,A040000165,0,2020-08-21 12:07:27,8836,NaN,4,4,0,2020-08-21 12:05:16,2020-08-21 12:07:27,A04,165,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523808,7281,A040165004,A040000165,1,2020-10-19 13:16:26,8836,NaN,4,4,0,2020-10-19 13:12:07,2020-10-19 13:16:26,A04,165,4
2525116,7349,A040165004,A040000165,0,2020-09-04 10:47:36,8836,NaN,4,4,0,2020-09-04 10:46:31,2020-09-04 10:47:36,A04,165,4
2525689,7381,A040165004,A040000165,0,2020-08-20 03:19:08,8836,NaN,4,4,0,2020-08-20 03:14:54,2020-08-20 03:19:08,A04,165,4
2526488,7428,A040165004,A040000165,0,2020-09-04 08:56:24,8836,NaN,4,4,0,2020-09-04 08:54:38,2020-09-04 08:56:24,A04,165,4


In [ ]:
time_df = df.groupby('userID').elapsedTime.rolling(window=3).sum()
time_df = time_df.reset_index()[['userID', 'elapsedTime']]
cond1 = time_df.elapsedTime >= 0
cond2 = time_df.elapsedTime < 3

df.loc[cond1 & cond2, 'answerCode'] = 0

In [ ]:
time_df = df.groupby('userID').elapsedTime.rolling(window=3).sum()
time_df = time_df.reset_index()[['userID', 'elapsedTime']]
cond1 = time_df.elapsedTime >= 0
cond2 = time_df.elapsedTime < 3

df.loc[cond1 & cond2, 'answerCode'] = 0

In [64]:
def outlier_remove_mean(data):
    threshold = 3
    mean = np.mean(data)
    std = np.std(data)
    if std == 0:
        return np.NaN
    z_scores = [(i - mean)/std for i in data]
    return np.mean(data[np.abs(z_scores) < threshold])